# Julia via juliacall Example

Demonstrates calling Julia from Python for performance-critical computations.

In [ ]:
study = "julia_demo"
data_file = "data/panel_data.csv"
out_fig = "output/figures/julia_demo.pdf"
out_table = "output/tables/julia_demo.tex"
out_meta = "output/provenance/julia_demo.yml"

In [ ]:
from pathlib import Path

import matplotlib
import numpy as np
import pandas as pd

matplotlib.use("Agg")
import matplotlib.pyplot as plt
from repro_tools import write_build_record

for p in [Path(out_fig), Path(out_table), Path(out_meta)]:
    p.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
# Import Julia and load Statistics
from juliacall import Main as jl

jl.seval("using Statistics")  # Load Statistics into Main
print(f"Julia version: {jl.VERSION}")
print("✓ Statistics loaded")

In [ ]:
# Load data
df = pd.read_csv(data_file)
print(f"Loaded {len(df)} rows")

In [ ]:
# Use Julia for fast computation
numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

# Example: Compute means in Julia (faster for large arrays)
stats = {}
for col in numeric_cols:
    arr = np.array(df[col])
    # Pass to Julia and compute
    julia_mean = jl.mean(arr)
    julia_std = jl.std(arr)
    stats[col] = {"mean": float(julia_mean), "std": float(julia_std)}

stats_df = pd.DataFrame(stats).T
print("\nStatistics computed in Julia:")
print(stats_df)

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 6))
stats_df["mean"].plot(kind="bar", ax=ax, yerr=stats_df["std"])
ax.set_title("Variable Means (computed in Julia)")
ax.set_ylabel("Mean ± SD")
plt.tight_layout()
fig.savefig(out_fig, dpi=300)
plt.close(fig)
print(f"✓ Saved {out_fig}")

In [ ]:
with open(out_table, "w") as f:
    f.write(stats_df.to_latex(float_format="%.3f"))
print(f"✓ Saved {out_table}")

In [ ]:
write_build_record(
    out_meta=Path(out_meta),
    artifact_name=study,
    command=["papermill", "notebooks/julia_demo.ipynb"],
    repo_root=Path(".").resolve(),
    inputs=[Path(data_file)],
    outputs=[Path(out_fig), Path(out_table)],
)
print(f"✓ Saved {out_meta}")
print("🎉 Complete!")